In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
customer = pd.read_csv('bigdata.csv',low_memory = False)

In [ ]:
# 202006~202105 있으나 일단 202105만
customer_use = customer[['DayTrading비중_202105','Swing비중_202105','Buy&Hold비중_202105',
                         '주거래상품_202105','선호시장_202105',
                         '시총1천억미만선호도_202105','시총1천억이상3천억미만선호도_202105',
                         '시총3천억이상1조미만선호도_202105','시총1조이상선호도_202105']]

In [ ]:
customer_use = customer_use.rename(columns={'DayTrading비중_202105':'Day','Swing비중_202105':'Swing','Buy&Hold비중_202105':'Hold' })
customer_use

In [ ]:
# 0, 0, 0인 데이터
test_mask = (customer_use.Day == 0) & (customer_use.Swing == 0) & (customer_use.Hold == 0)
test_data = customer_use.loc[test_mask,:]
print('0,0,0: \t'+str(len(test_data)))
# 아닌 데이터
mask = (customer_use.Day != 0) | (customer_use.Swing != 0) | (customer_use.Hold != 0)
use_data = customer_use.loc[mask,:]
print('no 0,0,0: \t'+str(len(use_data)))
print('총 갯수: \t'+str(len(test_data)+len(use_data)))

In [ ]:
customer_use = use_data

In [ ]:
use_data = use_data[['Day']]
use_data

# 엘보우 방법을 사용해서 최적의 클러스터 개수 찾기
distortions = []
for i in tqdm(range(1, 11)):
    km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=300, random_state=0)
    km.fit(customer_use)
    distortions.append(km.inertia_)
    
plt.plot(range(1,11), distortions, marker ='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.tight_layout()
plt.show()

In [ ]:
# km = KMeans(n_clusters=4, init='k-means++', n_init=10, max_iter=300, tol=1e-04, random_state=0)
km = KMeans(n_clusters=2, init='k-means++', n_init=10, max_iter=300, tol=1e-04, random_state=0)
y_km = km.fit_predict(use_data)
customer_use["cluster_id"] = km.labels_
customer_use

In [ ]:
customer_use.to_csv("target_customer.csv", encoding="utf-8-sig")

x = customer_use['Day']
color = customer_use['cluster_id']
plt.scatter(x, x, c=color)
plt.show()

mask0 = (customer_use.cluster_id == 0)
data0 = customer_use.loc[mask0,:]
print('/////////클러스터0//////////')
print('MEAN: \t'+str(data0['Day'].mean()))
print('STD: \t'+str(data0['Day'].std()))
print('MIN: \t'+str(data0['Day'].min()))
print('MAX: \t'+str(data0['Day'].max()))
print('NUM: \t'+str(len(data0)))
mask1 = (customer_use.cluster_id == 1)
data1 = customer_use.loc[mask1,:]
print('/////////클러스터1//////////')
print('MEAN: \t'+str(data1['Day'].mean()))
print('STD: \t'+str(data1['Day'].std()))
print('MIN: \t'+str(data1['Day'].min()))
print('MAX: \t'+str(data1['Day'].max()))
print('NUM: \t'+str(len(data1)))
# mask2 = (customer_use.cluster_id == 2)
# data2 = customer_use.loc[mask2,:]
# print('/////////클러스터2//////////')
# print('MEAN: \t'+str(data2['Day'].mean()))
# print('STD: \t'+str(data2['Day'].std()))
# print('MIN: \t'+str(data2['Day'].min()))
# print('MAX: \t'+str(data2['Day'].max()))
# print('NUM: \t'+str(len(data2)))
# mask3 = (customer_use.cluster_id == 3)
# data3 = customer_use.loc[mask3,:]
# print('/////////클러스터3//////////')
# print('MEAN: \t'+str(data3['Day'].mean()))
# print('STD: \t'+str(data3['Day'].std()))
# print('MIN: \t'+str(data3['Day'].min()))
# print('MAX: \t'+str(data3['Day'].max()))
# print('NUM: \t'+str(len(data3)))

plt.hist(customer_use['Day'])
plt.show()

# 실루엣 그래프로 군집 품질을 정량화 
km = KMeans(n_clusters=4, init='k-means++', n_init=10, max_iter=300, tol=1e-04, random_state=0)
y_km = km.fit_predict(customer_use)

from matplotlib import cm
from sklearn.metrics import silhouette_samples

cluster_labels = np.unique(y_km)
n_clusters = cluster_labels.shape[0]
silhouette_vals = silhouette_samples(customer_use, y_km, metric='euclidean')
y_ax_lower, y_ax_upper = 0, 0
yticks = []
for i, c in tqdm(enumerate(cluster_labels)):
    c_silhouette_vals = silhouette_vals[y_km == c]
    c_silhouette_vals.sort()
    y_ax_upper += len(c_silhouette_vals)
    color = cm.jet(float(i) / n_clusters)
    plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, edgecolor='none', color=color)
    yticks.append((y_ax_lower + y_ax_upper) / 2.)
    y_ax_lower += len(c_silhouette_vals)

silhouette_avg = np.mean(silhouette_vals)
plt.axvline(silhouette_avg, color='red', linestyle='--')
plt.yticks(yticks, cluster_labels +1)
plt.ylabel('Cluster')
plt.xlabel('Silhouette coefficient')
plt.tight_layout()
plt.show()

In [2]:
analysis_table = pd.read_csv("target_customer.csv", encoding="utf-8-sig")
analysis_table

,Unnamed: 0,Day,Swing,Hold,주거래상품_202105,선호시장_202105,시총1천억미만선호도_202105,시총1천억이상3천억미만선호도_202105,시총3천억이상1조미만선호도_202105,시총1조이상선호도_202105,cluster_id
0,1,0.0,0.0,100.0,주식,코스피,0.0,0.0,0.0,0.0,1
1,2,1.0,97.0,0.0,주식,코스피,0.0,4.0,13.0,82.0,1
2,3,3.0,94.0,1.0,주식,코스피,1.0,8.0,26.0,66.0,1
3,4,0.0,85.0,14.0,주식,코스피,0.0,0.0,0.0,100.0,1
4,5,38.0,59.0,1.0,주식,코스피,8.0,16.0,12.0,64.0,0
...,...,...,...,...,...,...,...,...,...,...,...
806376,1499980,0.0,0.0,100.0,펀드,코스피,0.0,0.0,0.0,0.0,1
806377,1499987,7.0,82.0,8.0,주식,코스닥,0.0,57.0,24.0,19.0,1
806378,1499988,10.0,85.0,3.0,주식,코스피,19.0,68.0,11.0,2.0,1
806379,1499990,0.0,91.0,8.0,퇴직연금,코스피,0.0,1.0,24.0,75.0,1


In [8]:
analysis_table['타겟고객'] = ['O' if x ==1
                          else 'X' for x in analysis_table['cluster_id']]

In [9]:
highrisk = ['주식','선물옵션','파생결합상품','신용공여']
midrisk = ['해당없음','펀드','랩','신탁','기타','상품현물']
lowrisk = ['채권','보험','퇴직연금','CMA/RP','예수금','발행어음']
analysis_table['위험선호도'] = ['고위험' if x in highrisk
                           else '중위험' if x in midrisk
                           else '저위험'
                           for x in analysis_table['주거래상품_202105']]

In [10]:
analysis_table['시장선호도'] = analysis_table['선호시장_202105']

In [11]:
analysis_table['시총선호도'] = ['X' if (w==0 and x ==0 and y==0 and z==0)
                           else '1조이상' if max(w,x,y,z)==w
                           else '1조미만'
                           for w,x,y,z in zip(analysis_table['시총1조이상선호도_202105'],
                                              analysis_table['시총3천억이상1조미만선호도_202105'],
                                              analysis_table['시총1천억이상3천억미만선호도_202105'],
                                              analysis_table['시총1천억미만선호도_202105'])]

In [12]:
analysis_table['주식'] = ['코스피대형주' if (x =='코스피' and y =='1조이상')
                       else '코스피중형주' if (x =='코스피' and y =='1조미만')
                       else '코스피' if (x =='코스피' and y =='X')
                       else '코스닥150' if x =='코스닥'
                       else '코스피/코스닥150'
                       for x,y in zip(analysis_table['시장선호도'],
                                      analysis_table['시총선호도'])]

In [13]:
analysis_table['장기채']= ['중장기국채' if x == '고위험' else '중장기국채' if x =='중위험' else '국채3년' for x in analysis_table['위험선호도']]

In [14]:
analysis_table['중기채'] = ['단기채권액티브' if x == '고위험' else '국채3년' if x =='중위험' else '단기통안채' for x in analysis_table['위험선호도']]

In [15]:
analysis_table['원자재']= ['금속선물/농산물선물/원유선물' for __ in analysis_table['주거래상품_202105']]

In [16]:
analysis_table['금']  = ['금은선물' for __ in analysis_table['주거래상품_202105']]

In [19]:
analysis_table.to_csv("customer_summary.csv", encoding="utf-8-sig")

In [31]:
analysis_table = pd.read_csv("../data/customer_summary.csv", encoding="utf-8-sig")
number = 1499989
if number in analysis_table['Unnamed: 0']:
    index = list(analysis_table['Unnamed: 0']).index(number)
    if(analysis_table['타겟고객'][index] == 'O'):
        print(f'{number}번 고객님은 AWP 타겟 고객입니다.')
        data = analysis_table['위험선호도'][index]
        print(f'위험선호도:\t{data}')
        data = analysis_table['시장선호도'][index]
        print(f'시장선호도:\t{data}')
        data = analysis_table['시총선호도'][index]
        print(f'시총선호도:\t{data}')
        data = analysis_table['주식'][index]
        print(f'주식:\t\t{data}')
        data = analysis_table['장기채'][index]
        print(f'장기채:\t\t{data}')
        data = analysis_table['중기채'][index]
        print(f'중기채:\t\t{data}')
        data = analysis_table['원자재'][index]
        print(f'원자재:\t\t{data}')
        data = analysis_table['금'][index]
        print(f'금:\t\t{data}')
    else:
        print(f'{number}번 고객님은 AWP 타겟 고객이 아닙니다.')
else:
    print(f'{number}번 고객님은 5월 거래데이터가 없어서 분석하지 못했습니다.')

1499989번 고객님은 5월 거래데이터가 없어서 분석하지 못했습니다.


In [28]:
analysis_table

,Unnamed: 0,Day,Swing,Hold,주거래상품_202105,선호시장_202105,시총1천억미만선호도_202105,시총1천억이상3천억미만선호도_202105,시총3천억이상1조미만선호도_202105,시총1조이상선호도_202105,cluster_id,타겟고객,위험선호도,시장선호도,시총선호도,주식,장기채,중기채,원자재,금
0,1,0.0,0.0,100.0,주식,코스피,0.0,0.0,0.0,0.0,1,O,고위험,코스피,X,코스피,중장기국채,단기채권액티브,금속선물/농산물선물/원유선물,금은선물
1,2,1.0,97.0,0.0,주식,코스피,0.0,4.0,13.0,82.0,1,O,고위험,코스피,1조이상,코스피대형주,중장기국채,단기채권액티브,금속선물/농산물선물/원유선물,금은선물
2,3,3.0,94.0,1.0,주식,코스피,1.0,8.0,26.0,66.0,1,O,고위험,코스피,1조이상,코스피대형주,중장기국채,단기채권액티브,금속선물/농산물선물/원유선물,금은선물
3,4,0.0,85.0,14.0,주식,코스피,0.0,0.0,0.0,100.0,1,O,고위험,코스피,1조이상,코스피대형주,중장기국채,단기채권액티브,금속선물/농산물선물/원유선물,금은선물
4,5,38.0,59.0,1.0,주식,코스피,8.0,16.0,12.0,64.0,0,X,고위험,코스피,1조이상,코스피대형주,중장기국채,단기채권액티브,금속선물/농산물선물/원유선물,금은선물
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806376,1499980,0.0,0.0,100.0,펀드,코스피,0.0,0.0,0.0,0.0,1,O,중위험,코스피,X,코스피,중장기국채,국채3년,금속선물/농산물선물/원유선물,금은선물
806377,1499987,7.0,82.0,8.0,주식,코스닥,0.0,57.0,24.0,19.0,1,O,고위험,코스닥,1조미만,코스닥150,중장기국채,단기채권액티브,금속선물/농산물선물/원유선물,금은선물
806378,1499988,10.0,85.0,3.0,주식,코스피,19.0,68.0,11.0,2.0,1,O,고위험,코스피,1조미만,코스피중형주,중장기국채,단기채권액티브,금속선물/농산물선물/원유선물,금은선물
806379,1499990,0.0,91.0,8.0,퇴직연금,코스피,0.0,1.0,24.0,75.0,1,O,저위험,코스피,1조이상,코스피대형주,국채3년,단기통안채,금속선물/농산물선물/원유선물,금은선물
